In [1]:
import os
import json, time, urllib.parse
import requests
from dotenv import load_dotenv
from loguru import logger
import pandas as pd
import numpy as np

### Set working directory

In [2]:
WD = '/Users/johnmichael/Documents/DATA512/data-512-homework_2'

### Import files

In [3]:
us_cities = pd.read_csv(os.path.join(WD, 'input/us_cities_by_state_SEPT.2023.csv'))
us_pop = pd.read_excel(os.path.join(WD, 'input/NST-EST2022-POP.xlsx'), skiprows=3)
us_regions = pd.read_excel(os.path.join(WD, 'input/US States by Region - US Census Bureau.xlsx'))

### Clean input files

In [4]:
'''
Clean regions file so that each row has a
unique region, division and state combination.
'''
us_regions_clean = us_regions.ffill()
us_regions_clean.dropna(subset='STATE', inplace=True)
us_regions_clean.drop_duplicates(subset='STATE', ignore_index=True, inplace=True)
us_regions_clean.columns = us_regions_clean.columns.str.lower()

In [5]:
'''
Clean US population data so that each row
contains a state and the 2022 population.
'''
us_pop_clean = us_pop.copy()
us_pop_clean.columns = ['state', 'pop_2020_est', 'pop_2020', 'pop_2021', 'pop_2022']
us_pop_clean = us_pop_clean[us_pop_clean.state.str.contains('^\.', na=False)]
us_pop_clean['state'] = us_pop_clean.state.str.slice(1)
us_pop_clean = us_pop_clean[['state', 'pop_2022']].reset_index(drop=True)

### Request page info

In [7]:
# Drop duplicate rows
us_cities.drop_duplicates(inplace=True, ignore_index=True)

In [8]:
'''
Any API-related code below are adopted from wp_page_info_example.ipynb
and wp_ores_liftwing_example.ipynb. Both notebooks are in the repository
https://github.com/jmic94/data-512-homework_2.
See the notebooks for full CC-BY license information.
'''

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = 'https://en.wikipedia.org/w/api.php'

'''
We'll assume that there needs to be some throttling for these requests - 
we should always be nice to a free data resource
'''
API_LATENCY_ASSUMED = 0.002 # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (20.0/100.0)-API_LATENCY_ASSUMED

'''
When making automated requests we should include something that is unique
to the person making the request. This should include an email - your UW email
would be good to put in there.
'''
REQUEST_HEADERS = {
    'User-Agent': '<jmic94@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles we are requesting info for
ARTICLE_TITLES = list(us_cities.page_title)

'''
This is a string of additional page properties that can be returned.
See the Info documentation for what can be included.
If you don't want any this can simply be the empty string.
'''
PAGEINFO_EXTENDED_PROPERTIES = 'talkid|url|watched|watchers'
# PAGEINFO_EXTENDED_PROPERTIES = ''

# This template lists the basic parameters for making this request
PAGEINFO_PARAMS_TEMPLATE = {
    'action': 'query',
    'format': 'json',
    'titles': '', # to simplify this should be a single page title at a time
    'prop': 'info',
    'inprop': PAGEINFO_EXTENDED_PROPERTIES
}

In [9]:
def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception('Must supply an article title to make a pageinfo request.')

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [ ]:
start = time.time()
page_df = pd.DataFrame()

In [ ]:
# Make page info requests
# Create log file to track progress
log_file_name = os.path.join(WD, 'intermediate/page_info_requests.log')
if os.path.exists(log_file_name):
    with open(log_file_name, 'w'):
        pass
logger.remove()
logger.add(log_file_name,
           format='{time:YYYY-MM-DD HH:mm:ss} | {message}',
           level='INFO',
           backtrace=False,
           diagnose=False)

for i, page in enumerate(ARTICLE_TITLES):
    logger.info(f'Progress: {i} - {page}')
    info = request_pageinfo_per_article(page)
    df = pd.DataFrame(info['query']['pages'].values())
    page_df = pd.concat([page_df, df])
logger.remove()

In [20]:
# Export page info request as csv
page_df.to_csv(os.path.join(WD, 'intermediate/page_info.csv'), index=False)
print(f'Elapsed time: {time.time() - start:.1f}s')

Elapsed time: 17116.7s


In [10]:
page_df = pd.read_csv(os.path.join(WD, 'intermediate/page_info.csv'))

### Request Article Quality Predictions

In [11]:
# Access token
'''
Store the access token in a .env file in a local clone of the repository.
This .env will be ignored by the repository and will not be pushed to the
remote Git.
'''
load_dotenv()
ACCESS_TOKEN = os.getenv('access_token')

# The current LiftWing ORES API endpoint and prediction model
API_ORES_LIFTWING_ENDPOINT = 'https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict'
API_ORES_EN_QUALITY_MODEL = 'enwiki-articlequality'

'''
The throttling rate is a function of the Access token that you are granted
when you request the token. The constants come from dissecting the token
and getting the rate limits from the granted token. An example of that is below.
'''
API_LATENCY_ASSUMED = 0.002 # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

'''
When making automated requests we should include something that is
unique to the person making the request. This should include an email
- your UW email would be good to put in there. Because all LiftWing
API requests require some form of authentication, you need to provide
your access token as part of the header too.
'''
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': '<jmic94@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

# This is a template for the parameters that we need to supply in the headers of an API request
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address': '', # your email address should go here
    'access_token': '' # the access token you create will need to go here
}

'''
A dictionary of English Wikipedia article titles (keys) and
sample revision IDs that can be used for this ORES scoring example.
'''
# ARTICLE_REVISIONS = {'Bison': 1085687913,
#                      'Northern flicker': 1086582504,
#                      'Red squirrel': 1083787665,
#                      'Chinook salmon': 1085406228,
#                      'Horseshoe bat': 1060601936}
# ARTICLE_REVISIONS = page_df[['title', 'lastrevid']].set_index('title')['lastrevid'].to_dict()


# This is a template of the data required as a payload when making a scoring request of the ORES model
ORES_REQUEST_DATA_TEMPLATE = {
    'lang': 'en', # required that its english - we're scoring English Wikipedia revisions
    'rev_id': '', # this request requires a revision id
    'features': True
}

In [12]:
def request_ores_score_per_article(article_revid = None,
                                   email_address=None,
                                   access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    # Make sure we have an article revision id, email and token
    # This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    # Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception('Must provide an article revision id (rev_id) to score articles')
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [57]:
start = time.time()
score_df = pd.DataFrame()
hparams = REQUEST_HEADER_PARAMS_TEMPLATE.copy()
hparams['email_address'] = "jmic94@uw.edu"
hparams['access_token'] = ACCESS_TOKEN

In [139]:
# Create log file to track progress
log_file_name = os.path.join(WD, 'intermediate/ores_requests.log')
if os.path.exists(log_file_name):
    with open(log_file_name, 'w'):
        pass
logger.remove()
logger.add(log_file_name,
           format='{time:YYYY-MM-DD HH:mm:ss} | {message}',
           level='INFO',
           backtrace=False,
           diagnose=False)

for i in range(page_df.shape[0]):
    title = page_df.title[i]
    rev_id = int(page_df.lastrevid[i])
    logger.info(f'Progress: {i} - {title}')
    rd = ORES_REQUEST_DATA_TEMPLATE.copy()
    rd['rev_id'] = rev_id
    score = request_ores_score_per_article(request_data=rd,
                                           header_params=hparams)
    quality = score['enwiki']['scores'][str(rev_id)][
        'articlequality']['score']['prediction']
    df = pd.DataFrame({'title': [title],
                       'rev_id': [rev_id],
                       'quality': [quality]})
    score_df = pd.concat([score_df, df])

In [140]:
score_df.to_csv(os.path.join(WD, 'intermediate/scores.csv'), index=False)
logger.remove()
print(f'Elapsed time: {time.time() - start:.1f}s')

Elapsed time: 52723.2s


In [13]:
score_df = pd.read_csv(os.path.join(WD, 'intermediate/scores.csv'))

### Combine data sets

In [14]:
combined_df = page_df.copy()

# Get state column
combined_df = combined_df.merge(us_cities, how='left',
                                left_on='title', right_on='page_title')

# Clean state names
combined_df['state'] = np.where(combined_df['state'] == 'Georgia_(U.S._state)',
                                'Georgia', combined_df['state'])
combined_df['state'] = combined_df.state.str.replace('_', ' ')

# Merge with scores data
combined_df = combined_df.merge(score_df[['rev_id', 'quality']],
                                left_on='lastrevid',
                                right_on='rev_id',
                                how='left')

# Merge with regional division data
combined_df = combined_df.merge(us_regions_clean, how='left', on='state')

# Merge with population data
combined_df = combined_df.merge(us_pop_clean, how='left', on='state')

# Rename columns
combined_df.rename(columns={'title': 'article_title',
                            'lastrevid': 'revision_id',
                            'quality': 'article_quality',
                            'division': 'regional_division',
                            'pop_2022': 'population'},
                   inplace=True)
combined_df = combined_df[['state', 'regional_division', 'population',
                           'article_title', 'revision_id', 'article_quality']]
combined_df.drop_duplicates(inplace=True, ignore_index=True)

# Export combined data set
combined_df.to_csv(os.path.join(WD, 'output/wp_scored_city_articles_by_state.csv'),
                   index=False)

### Create analysis tables

In [15]:
state_per_pop = combined_df.groupby('state')['article_title'].count().reset_index()
state_per_pop.rename(columns={'article_title': 'n_articles'}, inplace=True)
state_per_pop = state_per_pop.merge(combined_df[['state', 'population']],
                                    how='left', on='state')
state_per_pop.drop_duplicates(inplace=True, ignore_index=True)
state_per_pop['articles_per_pop'] = state_per_pop.n_articles / state_per_pop.population
state_per_pop.sort_values(by='articles_per_pop', ascending=False, inplace=True)

#### Top 10 US States by Article Coverage

In [16]:
state_per_pop.head(10).reset_index(drop=True)

,state,n_articles,population,articles_per_pop
0,Vermont,329,647064.0,0.000508
1,North Dakota,356,779261.0,0.000457
2,Maine,483,1385340.0,0.000349
3,South Dakota,311,909824.0,0.000342
4,Iowa,1043,3200517.0,0.000326
5,Alaska,149,733583.0,0.000203
6,Pennsylvania,2556,12972008.0,0.000197
7,Michigan,1773,10034113.0,0.000177
8,Wyoming,99,581381.0,0.000170
9,New Hampshire,234,1395231.0,0.000168


#### Bottom 10 US States by Article Coverage

In [17]:
state_per_pop.tail(10).sort_values(by='articles_per_pop').reset_index(drop=True)

,state,n_articles,population,articles_per_pop
0,North Carolina,50,10698973.0,0.000005
1,Nevada,19,3177772.0,0.000006
2,California,482,39029342.0,0.000012
3,Arizona,91,7359197.0,0.000012
4,Virginia,133,8683619.0,0.000015
5,Florida,412,22244823.0,0.000019
6,Oklahoma,75,4019800.0,0.000019
7,Kansas,63,2937150.0,0.000021
8,Maryland,157,6164660.0,0.000025
9,Wisconsin,192,5892539.0,0.000033


In [18]:
state_per_pop_hq = combined_df.copy()
state_per_pop_hq['hq'] = np.where(state_per_pop_hq['article_quality'].isin(['FA', 'GA']), 1, 0)
state_per_pop_hq = state_per_pop_hq.groupby('state')['hq'].sum().reset_index()
state_per_pop_hq.rename(columns={'hq': 'n_hq_articles'}, inplace=True)
state_per_pop_hq = state_per_pop_hq.merge(combined_df[['state', 'population']],
                                    how='left', on='state')
state_per_pop_hq.drop_duplicates(inplace=True, ignore_index=True)
state_per_pop_hq['hq_articles_per_pop'] = state_per_pop_hq.n_hq_articles / state_per_pop_hq.population
state_per_pop_hq.sort_values(by='hq_articles_per_pop', ascending=False, inplace=True)

#### Top 10 US States by High Quality Article Coverage

In [19]:
# hq = High Quality
state_per_pop_hq.head(10)

,state,n_hq_articles,population,hq_articles_per_pop
42,Vermont,45,647064.0,0.000070
47,Wyoming,39,581381.0,0.000067
38,South Dakota,56,909824.0,0.000062
45,West Virginia,106,1775156.0,0.000060
24,Montana,55,1122867.0,0.000049
26,New Hampshire,63,1395231.0,0.000045
35,Pennsylvania,566,12972008.0,0.000044
23,Missouri,263,6177957.0,0.000043
1,Alaska,31,733583.0,0.000042
27,New Jersey,379,9261699.0,0.000041


#### Bottom 10 US States by High Quality Article Coverage

In [20]:
state_per_pop_hq.tail(10).sort_values(by='hq_articles_per_pop').reset_index(drop=True)

,state,n_hq_articles,population,hq_articles_per_pop
0,North Carolina,20,10698973.0,0.000002
1,Virginia,18,8683619.0,0.000002
2,Nevada,8,3177772.0,0.000003
3,Arizona,24,7359197.0,0.000003
4,California,172,39029342.0,0.000004
5,Florida,119,22244823.0,0.000005
6,New York,111,19677151.0,0.000006
7,Maryland,42,6164660.0,0.000007
8,Kansas,22,2937150.0,0.000007
9,Oklahoma,31,4019800.0,0.000008


In [32]:
div_per_pop = combined_df.groupby('regional_division')['article_title'].count().reset_index()
div_pops = combined_df[['state', 'regional_division', 'population']].drop_duplicates()
div_pops = div_pops.groupby('regional_division')['population'].sum().reset_index()
div_per_pop.rename(columns={'article_title': 'n_articles'}, inplace=True)
div_per_pop = div_per_pop.merge(div_pops, how='left', on='regional_division')
div_per_pop.drop_duplicates(inplace=True, ignore_index=True)
div_per_pop['articles_per_pop'] = div_per_pop.n_articles / div_per_pop.population
div_per_pop.sort_values(by='articles_per_pop', ascending=False, inplace=True, ignore_index=True)

#### US Regional Divisions by Article Coverage

In [33]:
div_per_pop.head(10)

,regional_division,n_articles,population,articles_per_pop
0,West North Central,3578,19721893.0,0.000181
1,New England,1437,11503343.0,0.000125
2,East North Central,4754,47097779.0,0.000101
3,Middle Atlantic,3781,41910858.0,0.000090
4,East South Central,1529,19578002.0,0.000078
5,West South Central,2103,41685250.0,0.000050
6,Mountain,1189,25514320.0,0.000047
7,South Atlantic,1850,66781137.0,0.000028
8,Pacific,1304,53229044.0,0.000024


In [34]:
div_per_pop_hq = combined_df.copy()
div_per_pop_hq['hq'] = np.where(div_per_pop_hq['article_quality'].isin(['FA', 'GA']), 1, 0)
div_per_pop_hq = div_per_pop_hq.groupby('regional_division')['hq'].sum().reset_index()
div_per_pop_hq.rename(columns={'hq': 'n_hq_articles'}, inplace=True)
div_per_pop_hq = div_per_pop_hq.merge(div_pops[['regional_division', 'population']],
                                how='left', on='regional_division')
div_per_pop_hq.drop_duplicates(inplace=True, ignore_index=True)
div_per_pop_hq['hq_articles_per_pop'] = div_per_pop_hq.n_hq_articles / div_per_pop_hq.population
div_per_pop_hq.sort_values(by='hq_articles_per_pop', ascending=False, inplace=True, ignore_index=True)

#### US Regional Divisions by High Quality Article Coverage

In [35]:
div_per_pop_hq

,regional_division,n_hq_articles,population,hq_articles_per_pop
0,West North Central,640,19721893.0,0.000032
1,Middle Atlantic,1056,41910858.0,0.000025
2,New England,225,11503343.0,0.000020
3,East South Central,317,19578002.0,0.000016
4,East North Central,717,47097779.0,0.000015
5,West South Central,634,41685250.0,0.000015
6,Mountain,336,25514320.0,0.000013
7,Pacific,489,53229044.0,0.000009
8,South Atlantic,526,66781137.0,0.000008
